<hr style="height:.9px;border:none;color:#333;background-color:#333;" />
<hr style="height:.9px;border:none;color:#333;background-color:#333;" />

<br><h2>  Assignment Classification model development model
    <h4> By | Hasnain Ahmed</h4>


Machine learning <br>

<em><strong>Hult International Business School</strong></em><br>
<br>


Machine Learning<br><br><br>
<hr style="height:1px;border:none;color:#333;background-color:#333;" />
<hr style="height:1px;border:none;color:#333;background-color:#333;" />
<br><br><br>



In [393]:
"""
    DocString:
    
    1) PROCESS:-
    
    A) The following is a machine learning, predictive model made by Hasnain Ahmed using the Apprentice chef 
       data base.Here we seek to make a classification model on our data to find the best possible using the 
       the methodlogies of ols regressioin and lasso for the dependant variable called cross selling seccess 
       where we are atampting to bild the best AUC model as well as a GBM tree with the use of hyperperameter tuning.
       
       
    B)we use a bit of the feature engineering carried forward from the regression model to find the best variables 
        to give us the best results without eliminating too many variables as well as filling in/dropping all 
        nan variables to better build a linear  model and hopefully looking for the best AUC score.
    


    2) Known Bugs and/or Errors:
    None Found.
    
"""


'\n    DocString:\n    \n    1) PROCESS:-\n    \n    A) The following is a machine learning, predictive model made by Hasnain Ahmed using the Apprentice chef \n       data base.Here we seek to make a classification model on our data to find the best possible using the \n       the methodlogies of ols regressioin and lasso for the dependant variable called cross selling seccess \n       where we are atampting to bild the best AUC model as well as a GBM tree with the use of hyperperameter tuning.\n       \n       \n    B)we use a bit of the feature engineering carried forward from the regression model to find the best variables \n        to give us the best results without eliminating too many variables as well as filling in/dropping all \n        nan variables to better build a linear  model and hopefully looking for the best AUC score.\n    \n\n\n    2) Known Bugs and/or Errors:\n    None Found.\n    \n'

In [53]:
# importing libraries
import random            as rand   # random number gen
import pandas            as pd                       # data science essentials
import matplotlib.pyplot as plt                      # data visualization
import seaborn           as sns                      # enhanced data viz
from sklearn.model_selection import train_test_split # train-test split
from sklearn.linear_model import LogisticRegression  # logistic regression
import statsmodels.formula.api as smf                # logistic regression
from sklearn.metrics import confusion_matrix         # confusion matrix
from sklearn.metrics import roc_auc_score            # auc score
from sklearn.neighbors import KNeighborsClassifier   # KNN for classification
from sklearn.neighbors import KNeighborsRegressor    # KNN for regression
import operator #important package to remove p-values
import statsmodels.api as sm #important for removing p-values
from sklearn.preprocessing import StandardScaler     # standard scaler
from sklearn.linear_model import LinearRegression
# new packages
from sklearn.ensemble import RandomForestClassifier     # random forest
from sklearn.ensemble import GradientBoostingClassifier # gbm
# CART model packages
from sklearn.tree import DecisionTreeClassifier      # classification trees
from sklearn.tree import export_graphviz             # exports graphics
from six import StringIO           # saves objects in memory
from IPython.display import Image                    # displays on frontend
import pydotplus                                     # interprets dot objects
import graphviz 

from sklearn import tree
# libraries for hyperparameter tuning
from sklearn.model_selection import RandomizedSearchCV     # hyperparameter tuning
from sklearn.metrics import make_scorer                    # customizable scorer



#importing the data set 
file = "Apprentice_Chef_Dataset.xlsx"


# setting pandas print options
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)
pd.set_option('display.max_colwidth', 100)





# reading the file into Python
chef = pd.read_excel(io=file)
# displaying teh raw data 
chef.head(n=5)


,REVENUE,CROSS_SELL_SUCCESS,NAME,EMAIL,FIRST_NAME,FAMILY_NAME,TOTAL_MEALS_ORDERED,UNIQUE_MEALS_PURCH,CONTACTS_W_CUSTOMER_SERVICE,PRODUCT_CATEGORIES_VIEWED,AVG_TIME_PER_SITE_VISIT,MOBILE_NUMBER,CANCELLATIONS_BEFORE_NOON,CANCELLATIONS_AFTER_NOON,TASTES_AND_PREFERENCES,PC_LOGINS,MOBILE_LOGINS,WEEKLY_PLAN,EARLY_DELIVERIES,LATE_DELIVERIES,PACKAGE_LOCKER,REFRIGERATED_LOCKER,AVG_PREP_VID_TIME,LARGEST_ORDER_SIZE,MASTER_CLASSES_ATTENDED,MEDIAN_MEAL_RATING,AVG_CLICKS_PER_VISIT,TOTAL_PHOTOS_VIEWED
0,393.0,1,Saathos,saathos@unitedhealth.com,Saathos,Saathos,14,6,12,10,48.00,1,3,1,1,5,2,0,0,2,0,0,33.4,1,0,1,17,0
1,1365.0,1,Alysanne Osgrey,alysanne.osgrey@ge.org,Alysanne,Osgrey,87,3,8,8,40.35,1,0,0,1,5,1,12,0,2,0,0,84.8,1,0,3,13,170
2,800.0,1,Edwyd Fossoway,edwyd.fossoway@jnj.com,Edwyd,Fossoway,15,7,11,5,19.77,1,3,0,1,6,1,1,0,1,0,0,63.0,1,0,2,16,0
3,600.0,1,Eleyna Westerling,eleyna.westerling@ge.org,Eleyna,Westerling,13,6,11,5,90.00,1,2,0,1,6,1,14,0,3,0,0,43.8,1,0,2,14,0
4,1490.0,1,Elyn Norridge,elyn.norridge@jnj.com,Elyn,Norridge,47,8,6,10,40.38,1,0,0,0,5,1,5,0,8,0,0,84.8,1,1,3,12,205


In [54]:
#Feature Engineering

#Setting outlier boundaries for relevant variables (enough observations)

AVG_TIME_PER_SITE_VISIT_high = 200      #Set boundary
AVG_PREP_VID_TIME_high = 300            #Set boundary
TOTAL_MEALS_ORDERED_high = 250          #Set boundary
WEEKLY_PLAN_high = 15                   #Set boundary
TOTAL_PHOTOS_VIEWED_high = 600          #Set boundary
UNIQUE_MEALS_PURCH_high = 10            #Set boundary 
CONTACTS_W_CUSTOMER_SERVICE_low = 2     #Set boundary 
CONTACTS_W_CUSTOMER_SERVICE_high = 13   #Set boundary
CANCELLATIONS_BEFORE_NOON_high = 6      #Set boundary
EARLY_DELIVERIES_high = 5               #Set boundary
LATE_DELIVERIES_high = 10               #Set boundary
LARGEST_ORDER_SIZE_low = 1              #Set boundary
LARGEST_ORDER_SIZE_high = 8             #Set boundary
AVG_CLICKS_PER_VISIT_low = 8            #Set boundary
AVG_CLICKS_PER_VISIT_high = 18          #Set boundary

#creating new columns with outliers based on boundaries

# Average time per site visit
chef['Outlier_AVG_TIME_PER_SITE_VISIT'] = 0

chef.loc[chef.AVG_TIME_PER_SITE_VISIT >= AVG_TIME_PER_SITE_VISIT_high, 'Outlier_AVG_TIME_PER_SITE_VISIT'] = 1

# Average prep vid time
chef['Outlier_AVG_PREP_VID_TIME'] = 0

chef.loc[chef.AVG_PREP_VID_TIME >= AVG_PREP_VID_TIME_high, 'Outlier_AVG_PREP_VID_TIME'] = 1


# Total Meals ordered
chef['Outlier_TOTAL_MEALS_ORDERED'] = 0

chef.loc[chef.TOTAL_MEALS_ORDERED >= TOTAL_MEALS_ORDERED_high, 'Outlier_TOTAL_MEALS_ORDERED'] = 1


#Weekly Plan
chef['Outlier_WEEKLY_PLAN'] = 0

chef.loc[chef.WEEKLY_PLAN >= WEEKLY_PLAN_high, 'Outlier_WEEKLY_PLAN'] = 1


# Total photos viewed
chef['Outlier_TOTAL_PHOTOS_VIEWED'] = 0

chef.loc[chef.TOTAL_PHOTOS_VIEWED >= TOTAL_PHOTOS_VIEWED_high, 'Outlier_TOTAL_PHOTOS_VIEWED'] = 1


# Unique meals purchase
chef['Outlier_UNIQUE_MEALS_PURCH'] = 0

chef.loc[chef.UNIQUE_MEALS_PURCH >= UNIQUE_MEALS_PURCH_high, 'Outlier_UNIQUE_MEALS_PURCH'] = 1


# Cancellations after noon
chef['Outlier_CONTACTS_W_CUSTOMER_SERVICE'] = 0

chef.loc[chef.CONTACTS_W_CUSTOMER_SERVICE >= CONTACTS_W_CUSTOMER_SERVICE_high, 'Outlier_CONTACTS_W_CUSTOMER_SERVICE'] = 1
chef.loc[chef.CONTACTS_W_CUSTOMER_SERVICE <= CONTACTS_W_CUSTOMER_SERVICE_low, 'Outlier_CONTACTS_W_CUSTOMER_SERVICE'] = 1


# Cancellations before noon
chef['Outlier_CANCELLATIONS_BEFORE_NOON'] = 0

chef.loc[chef.CANCELLATIONS_BEFORE_NOON >= CANCELLATIONS_BEFORE_NOON_high, 'Outlier_CANCELLATIONS_BEFORE_NOON'] = 1


#Early Deliveries
chef['Outlier_EARLY_DELIVERIES'] = 0

chef.loc[chef.EARLY_DELIVERIES >= EARLY_DELIVERIES_high, 'Outlier_EARLY_DELIVERIES'] = 1


# Late Deliveries
chef['Outlier_LATE_DELIVERIES'] = 0

chef.loc[chef.LATE_DELIVERIES >= LATE_DELIVERIES_high, 'Outlier_LATE_DELIVERIES'] = 1


#Largest Order Size
chef['Outlier_LARGEST_ORDER_SIZE'] = 0

chef.loc[chef.LARGEST_ORDER_SIZE <= LARGEST_ORDER_SIZE_low, 'Outlier_LARGEST_ORDER_SIZE'] = 1
chef.loc[chef.LARGEST_ORDER_SIZE >= LARGEST_ORDER_SIZE_high, 'Outlier_LARGEST_ORDER_SIZE'] = 1


# Average clicks per visit
chef['Outlier_AVG_CLICKS_PER_VISIT'] = 0

chef.loc[chef.AVG_CLICKS_PER_VISIT <= AVG_CLICKS_PER_VISIT_low, 'Outlier_AVG_CLICKS_PER_VISIT'] = 1
chef.loc[chef.AVG_CLICKS_PER_VISIT >= AVG_CLICKS_PER_VISIT_high, 'Outlier_AVG_CLICKS_PER_VISIT'] = 1




In [55]:
###
#Variables with EDA again checked and some variables show high amount of 0s
#They will be also flagged below
####


#creating objects for every variable that contains a lot of times the value zero
CANCELLATIONS_BEFORE_NOON_zero = 0            #Set boundary 
CANCELLATIONS_AFTER_NOON_zero = 0             #Set boundary
WEEKLY_PLAN_zero = 0                          #Set boundary
TOTAL_PHOTOS_VIEWED_zero = 0                  #Set boundary
EARLY_DELIVERIES_zero = 0                     #Set boundary
LATE_DELIVERIES_zero = 0                      #Set boundary
MASTER_CLASSES_ATTENDED_zero = 0              #Set boundary

# Cancelations before noon
chef['Zero_CANCELLATIONS_BEFORE_NOON'] = 0
chef.loc[chef.CANCELLATIONS_BEFORE_NOON == CANCELLATIONS_BEFORE_NOON_zero, 'Zero_CANCELLATIONS_BEFORE_NOON'] = 1

# Cancelations after noon
chef['Zero_CANCELLATIONS_AFTER_NOON'] = 0
chef.loc[chef.CANCELLATIONS_AFTER_NOON == CANCELLATIONS_AFTER_NOON_zero, 'Zero_CANCELLATIONS_AFTER_NOON'] = 1

# Weekly Plan
chef['Zero_WEEKLY_PLAN'] = 0
chef.loc[chef.WEEKLY_PLAN == WEEKLY_PLAN_zero, 'Zero_WEEKLY_PLAN'] = 1

# Total Photos viewed
chef['Zero_TOTAL_PHOTOS_VIEWED'] = 0
chef.loc[chef.TOTAL_PHOTOS_VIEWED == TOTAL_PHOTOS_VIEWED_zero, 'Zero_TOTAL_PHOTOS_VIEWED'] = 1


# Early Delivery
chef['Zero_EARLY_DELIVERIES'] = 0
chef.loc[chef.EARLY_DELIVERIES == EARLY_DELIVERIES_zero, 'Zero_EARLY_DELIVERIES'] = 1


# Late Delivery
chef['Zero_LATE_DELIVERIES'] = 0
chef.loc[chef.LATE_DELIVERIES == LATE_DELIVERIES_zero, 'Zero_LATE_DELIVERIES'] = 1


# Master Class Attented
chef['Zero_MASTER_CLASSES_ATTENDED'] = 0
chef.loc[chef.MASTER_CLASSES_ATTENDED == MASTER_CLASSES_ATTENDED_zero, 'Zero_MASTER_CLASSES_ATTENDED'] = 1



In [56]:
####
#Transform some variables into logs based on EDA (if exp trend is visible)
####


#Transform Revenue in log and store result in new column
chef['log_Revenue'] = np.log10(chef['REVENUE'])

#Transform CONTACTS_W_CUSTOMER_SERVICE in log and store result in new column
chef['log_CONTACTS_W_CUSTOMER_SERVICE'] = np.log10(chef['CONTACTS_W_CUSTOMER_SERVICE'])

#Transform Total_Meals_Ordered in log and store result in new column
chef['log_Total_Meals_Ordered'] = np.log10(chef['TOTAL_MEALS_ORDERED'])

#Transform Total_Meals_Ordered in log and store result in new column
chef['log_LARGEST_ORDER_SIZE'] = np.log10(chef['LARGEST_ORDER_SIZE'])

#Transform Total_Meals_Ordered in log and store result in new column
chef['log_AVG_CLICKS_PER_VISIT'] = np.log10(chef['AVG_CLICKS_PER_VISIT'])

In [57]:
#####
#Create new variables and create ratios
####

#create new variable called "Single Meal" based on unique meals purchased

chef['Single_Meals'] = 0
chef.loc[chef['UNIQUE_MEALS_PURCH'] == 1, 'Single_Meals'] = 1

# total clicks on all devices
chef['TOTAL_CLICKS_ratio'] = chef['AVG_CLICKS_PER_VISIT'] * ( chef['PC_LOGINS'] + chef['MOBILE_LOGINS'] )

# Create new variable to combine pc and mobile logins to total logins
chef['TOTAL_LOGINS_all'] = chef['PC_LOGINS'] + chef['MOBILE_LOGINS']

# Create new variable Total Logins per total meals ordered
chef['Total_Logins_per_Total_Meals_Ordered'] = (chef.TOTAL_LOGINS_all) / chef.TOTAL_MEALS_ORDERED

# create new variable Categories viewed per total number of logins 
chef['Product_Categories_Viewed_Per_TOTAL_LOGINS'] = chef.PRODUCT_CATEGORIES_VIEWED / (chef.TOTAL_LOGINS_all)

#create new variable Unique Meals Purchases per Total Orders
chef['Unique_Meals_Purch_per_Total_Meals_Ordered'] = chef['UNIQUE_MEALS_PURCH'] / chef['TOTAL_MEALS_ORDERED']

#create new variable to create late delivery per total meals ordered
chef['Late_Delvery_per_Total_Meals'] = chef.LATE_DELIVERIES / chef.TOTAL_MEALS_ORDERED 

#create new variable to create early delivery per total meals ordered
chef['Early_Delvery_per_Total_Meals'] = chef.EARLY_DELIVERIES / chef.TOTAL_MEALS_ORDERED

# Create new variable to combine all wrong deliveries (Since too early and too late is not punctual)
chef['All_Wrong_Deliveries'] = chef['EARLY_DELIVERIES'] + chef['LATE_DELIVERIES']

# create new variable product categories viewed per total number of logins 
chef['Product_Categories_Viewed_per_Total_Logins'] = chef.PRODUCT_CATEGORIES_VIEWED / (chef.TOTAL_LOGINS_all)

#create new variable for contacts with customer service per total meals ordered
chef['Contacts_W_Customer_Service_per_Total_Orders'] = chef.CONTACTS_W_CUSTOMER_SERVICE /  chef.TOTAL_MEALS_ORDERED

#Feature Engineering for classification model

#Early Deliveries
chef['new_EARLY_DELIVERIES'] = np.where(chef.EARLY_DELIVERIES > 0,1,0)

#Late Deliveries
chef['new_LATE_DELIVERIES'] = np.where(chef.LATE_DELIVERIES > 0,1,0)

In [58]:
########
#Group Emails
#########

# STEP 1: splitting personal emails

# placeholder list
placeholder_l = []

# looping over each email address
for index, col in chef.iterrows():
    
    # splitting email domain at '@'
    email_split= chef.loc[index, 'EMAIL'].split(sep = '@')
   
    # appending placeholder_lst with the results
                                          
                           
    placeholder_l.append(email_split)
    

# converting placeholder_lst into a DataFrame 
df_emails = pd.DataFrame(placeholder_l)


# STEP 2: concatenating with original DataFrame
# renaming column to concatenate
df_emails.columns = ['0' , 'emails']


# concatenating personal emails  with friends DataFrame
chef = pd.concat([chef, df_emails['emails']],
                   axis = 1)

#creating email domain group objects
professional_emails= ['@mmm.com',
                               '@amex.com',
                               '@apple.com',
                               '@boeing.com',
                               '@caterpillar.com',
                               '@chevron.com',
                               '@cisco.com',
                               '@cocacola.com',
                               '@disney.com',
                               '@dupont.com',
                               '@exxon.com',
                               '@ge.org',
                               '@goldmansacs.com',
                               '@homedepot.com',
                               '@ibm.com',
                               '@intel.com',
                               '@jnj.com',
                               '@jpmorgan.com',
                               '@mcdonalds.com',
                               '@merck.com',
                               '@microsoft.com',
                               '@nike.com',
                               '@pfizer.com',
                               '@pg.com',
                               '@travelers.com',
                               '@unitedtech.com',
                               '@unitedhealth.com',
                               '@verizon.com',
                               '@visa.com',
                               '@walmart.com']

personal_emails = ['@gmail.com',
                         '@yahoo.com',
                         '@protonmail.com']

junk_emails = ['@me.com',
                      '@aol.com',
                      '@hotmail.com',
                      '@live.com',
                      '@msn.com',
                      '@passport.com']


# placeholder list
placeholder_l = []


# looping to group observations by domain type
for domain in chef['emails']:
        if '@' + domain in professional_emails:
            placeholder_l.append('professional')
            
        elif '@' + domain in personal_emails:
            placeholder_l.append('personal')
        
        elif '@' + domain in junk_emails:
            placeholder_l.append('junk')
            
        else:
            print('Unknown')


# concatenating with original DataFrame
chef['domain_group'] = pd.Series(placeholder_l)


# checking results
chef['domain_group'].value_counts()

personal        861
professional    696
junk            389
Name: domain_group, dtype: int64

In [59]:
#One hot Encoding

#Creating dummies for the variable email
dummy_email = pd.get_dummies(chef['domain_group'])

#Dropping the variables email and domain_group from our chef
chef = chef.drop('EMAIL', axis = 1)
chef = chef.drop('domain_group', axis = 1)

#Including the dummies in the chef
chef = chef.join([dummy_email])


#Storing variables in new object

# making a copy of chef
chef_copy = chef.copy()


# dropping SalePrice and Order from the explanatory variable set
chef_explain = chef_copy.drop([ 'junk',
                               'CROSS_SELL_SUCCESS',
                               'NAME',
                               'FIRST_NAME',
                                'FAMILY_NAME',
                                 'emails'],
                                               axis = 1)                
              
                                                



# formatting each explanatory variable for statsmodels
for val in chef_explain:
    print(val , '+')



REVENUE +
TOTAL_MEALS_ORDERED +
UNIQUE_MEALS_PURCH +
CONTACTS_W_CUSTOMER_SERVICE +
PRODUCT_CATEGORIES_VIEWED +
AVG_TIME_PER_SITE_VISIT +
MOBILE_NUMBER +
CANCELLATIONS_BEFORE_NOON +
CANCELLATIONS_AFTER_NOON +
TASTES_AND_PREFERENCES +
PC_LOGINS +
MOBILE_LOGINS +
WEEKLY_PLAN +
EARLY_DELIVERIES +
LATE_DELIVERIES +
PACKAGE_LOCKER +
REFRIGERATED_LOCKER +
AVG_PREP_VID_TIME +
LARGEST_ORDER_SIZE +
MASTER_CLASSES_ATTENDED +
MEDIAN_MEAL_RATING +
AVG_CLICKS_PER_VISIT +
TOTAL_PHOTOS_VIEWED +
Outlier_AVG_TIME_PER_SITE_VISIT +
Outlier_AVG_PREP_VID_TIME +
Outlier_TOTAL_MEALS_ORDERED +
Outlier_WEEKLY_PLAN +
Outlier_TOTAL_PHOTOS_VIEWED +
Outlier_UNIQUE_MEALS_PURCH +
Outlier_CONTACTS_W_CUSTOMER_SERVICE +
Outlier_CANCELLATIONS_BEFORE_NOON +
Outlier_EARLY_DELIVERIES +
Outlier_LATE_DELIVERIES +
Outlier_LARGEST_ORDER_SIZE +
Outlier_AVG_CLICKS_PER_VISIT +
Zero_CANCELLATIONS_BEFORE_NOON +
Zero_CANCELLATIONS_AFTER_NOON +
Zero_WEEKLY_PLAN +
Zero_TOTAL_PHOTOS_VIEWED +
Zero_EARLY_DELIVERIES +
Zero_LATE_DELIVERIES 

In [60]:
#Checking correlation
# creating a (Pearson) correlation matrix
df_corr = chef.corr(method = 'pearson').round(2)



# printing (Pearson) correlations with Revenue (sorted)
print(df_corr.loc['CROSS_SELL_SUCCESS'].sort_values(ascending = False))

CROSS_SELL_SUCCESS                              1.00
professional                                    0.19
CANCELLATIONS_BEFORE_NOON                       0.16
MOBILE_NUMBER                                   0.10
TASTES_AND_PREFERENCES                          0.08
REFRIGERATED_LOCKER                             0.07
Outlier_CANCELLATIONS_BEFORE_NOON               0.06
Zero_CANCELLATIONS_AFTER_NOON                   0.05
log_CONTACTS_W_CUSTOMER_SERVICE                 0.05
CONTACTS_W_CUSTOMER_SERVICE                     0.04
personal                                        0.04
PC_LOGINS                                       0.04
PACKAGE_LOCKER                                  0.04
MASTER_CLASSES_ATTENDED                         0.04
AVG_PREP_VID_TIME                               0.03
MEDIAN_MEAL_RATING                              0.03
Outlier_LATE_DELIVERIES                         0.03
log_LARGEST_ORDER_SIZE                          0.03
EARLY_DELIVERIES                              

In [61]:
# instantiating a logistic regression model object
logistic_small = smf.logit(formula = """CROSS_SELL_SUCCESS ~ REVENUE +
TOTAL_MEALS_ORDERED +
UNIQUE_MEALS_PURCH +
CONTACTS_W_CUSTOMER_SERVICE +
PRODUCT_CATEGORIES_VIEWED +
AVG_TIME_PER_SITE_VISIT +
MOBILE_NUMBER +
CANCELLATIONS_BEFORE_NOON +
CANCELLATIONS_AFTER_NOON +
TASTES_AND_PREFERENCES +
PC_LOGINS +
MOBILE_LOGINS +
WEEKLY_PLAN +
EARLY_DELIVERIES +
LATE_DELIVERIES +
PACKAGE_LOCKER +
REFRIGERATED_LOCKER +
AVG_PREP_VID_TIME +
LARGEST_ORDER_SIZE +
MASTER_CLASSES_ATTENDED +
MEDIAN_MEAL_RATING +
AVG_CLICKS_PER_VISIT +
TOTAL_PHOTOS_VIEWED +
Outlier_AVG_TIME_PER_SITE_VISIT +
Outlier_AVG_PREP_VID_TIME +
Outlier_TOTAL_MEALS_ORDERED +
Outlier_WEEKLY_PLAN +
Outlier_TOTAL_PHOTOS_VIEWED +
Outlier_UNIQUE_MEALS_PURCH +
Outlier_CONTACTS_W_CUSTOMER_SERVICE +
Outlier_CANCELLATIONS_BEFORE_NOON +
Outlier_EARLY_DELIVERIES +
Outlier_LATE_DELIVERIES +
Outlier_LARGEST_ORDER_SIZE +
Outlier_AVG_CLICKS_PER_VISIT +
Zero_CANCELLATIONS_BEFORE_NOON +
Zero_CANCELLATIONS_AFTER_NOON +
Zero_WEEKLY_PLAN +
Zero_TOTAL_PHOTOS_VIEWED +
Zero_EARLY_DELIVERIES +
Zero_LATE_DELIVERIES +
Zero_MASTER_CLASSES_ATTENDED +
log_Revenue +
log_CONTACTS_W_CUSTOMER_SERVICE +
log_Total_Meals_Ordered +
log_LARGEST_ORDER_SIZE +
log_AVG_CLICKS_PER_VISIT +
Single_Meals +
TOTAL_CLICKS_ratio +
TOTAL_LOGINS_all +
Total_Logins_per_Total_Meals_Ordered +
Product_Categories_Viewed_Per_TOTAL_LOGINS +
Unique_Meals_Purch_per_Total_Meals_Ordered +
Late_Delvery_per_Total_Meals +
Early_Delvery_per_Total_Meals +
All_Wrong_Deliveries +
Product_Categories_Viewed_per_Total_Logins +
Contacts_W_Customer_Service_per_Total_Orders +
personal +
professional 
                                                    
                                                """,
                                     data = chef)
                                                

# fitting the model object
results_logistic = logistic_small.fit()


# checking the results SUMMARY
results_logistic.summary()

         Current function value: 0.539570
         Iterations: 35


C:\Users\hp\anaconda3\lib\site-packages\statsmodels\base\model.py:566: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "


<class 'statsmodels.iolib.summary.Summary'>
"""
                           Logit Regression Results                           
==============================================================================
Dep. Variable:     CROSS_SELL_SUCCESS   No. Observations:                 1946
Model:                          Logit   Df Residuals:                     1888
Method:                           MLE   Df Model:                           57
Date:                Wed, 27 Jan 2021   Pseudo R-squ.:                  0.1405
Time:                        02:15:09   Log-Likelihood:                -1050.0
converged:                      False   LL-Null:                       -1221.6
Covariance Type:            nonrobust   LLR p-value:                 1.744e-42
================================================================================================================
                                                   coef    std err          z      P>|z|      [0.025      0.975]
----------------------------------------------------------------------------------------------------------------
Intercept                                       -6.9357     16.784     -0.413      0.679     -39.832      25.961
REVENUE                                      -3.238e-05      0.001     -0.045      0.964      -0.001       0.001
TOTAL_MEALS_ORDERED                             -0.0165      0.007     -2.216      0.027      -0.031      -0.002
UNIQUE_MEALS_PURCH                               0.0581      0.034      1.708      0.088      -0.009       0.125
CONTACTS_W_CUSTOMER_SERVICE                      0.2481        nan        nan        nan         nan         nan
PRODUCT_CATEGORIES_VIEWED                       -0.0116        nan        nan        nan         nan         nan
AVG_TIME_PER_SITE_VISIT                          0.0008      0.001      0.812      0.417      -0.001       0.003
MOBILE_NUMBER                                    0.8638      0.204      4.244      0.000       0.465       1.263
CANCELLATIONS_BEFORE_NOON                        0.3301      0.066      5.006      0.000       0.201       0.459
CANCELLATIONS_AFTER_NOON                        -0.1699      0.323     -0.526      0.599      -0.803       0.464
TASTES_AND_PREFERENCES                           0.4259      0.119      3.572      0.000       0.192       0.660
PC_LOGINS                                        0.1535        nan        nan        nan         nan         nan
MOBILE_LOGINS                                   -0.2752   6.49e+10  -4.24e-12      1.000   -1.27e+11    1.27e+11
WEEKLY_PLAN                                     -0.0080      0.010     -0.767      0.443      -0.029       0.012
EARLY_DELIVERIES                                 0.0951        nan        nan        nan         nan         nan
LATE_DELIVERIES                                 -0.0579        nan        nan        nan         nan         nan
PACKAGE_LOCKER                                   0.0023      0.124      0.019      0.985      -0.240       0.245
REFRIGERATED_LOCKER                              0.5219      0.222      2.349      0.019       0.087       0.957
AVG_PREP_VID_TIME                                0.0017      0.008      0.221      0.825      -0.013       0.017
LARGEST_ORDER_SIZE                              -0.1568      0.083     -1.896      0.058      -0.319       0.005
MASTER_CLASSES_ATTENDED                         -0.1117      0.199     -0.562      0.574      -0.502       0.278
MEDIAN_MEAL_RATING                              -0.0207      0.146     -0.142      0.887      -0.307       0.265
AVG_CLICKS_PER_VISIT                            -0.1125      0.656     -0.171      0.864      -1.399       1.174
TOTAL_PHOTOS_VIEWED                              0.0006      0.001      0.492      0.623      -0.002       0.003
Outlier_AVG_TIME_PER_SITE_VISIT                 -0.1888      0.390     -0.484      0.628      -0.953       0.575
Outlier_AVG_PREP_VID_TIME                        1.2456      0.747      1.668

In [62]:
# building a base model 

# INSTANTIATING a model type
lm_practice = smf.ols(formula = """CROSS_SELL_SUCCESS ~ REVENUE +
TOTAL_MEALS_ORDERED +
UNIQUE_MEALS_PURCH +
CONTACTS_W_CUSTOMER_SERVICE +
PRODUCT_CATEGORIES_VIEWED +
AVG_TIME_PER_SITE_VISIT +
MOBILE_NUMBER +
CANCELLATIONS_BEFORE_NOON +
CANCELLATIONS_AFTER_NOON +
TASTES_AND_PREFERENCES +
PC_LOGINS +
MOBILE_LOGINS +
WEEKLY_PLAN +
EARLY_DELIVERIES +
LATE_DELIVERIES +
PACKAGE_LOCKER +
REFRIGERATED_LOCKER +
AVG_PREP_VID_TIME +
LARGEST_ORDER_SIZE +
MASTER_CLASSES_ATTENDED +
MEDIAN_MEAL_RATING +
AVG_CLICKS_PER_VISIT +
TOTAL_PHOTOS_VIEWED +
Outlier_AVG_TIME_PER_SITE_VISIT +
Outlier_AVG_PREP_VID_TIME +
Outlier_TOTAL_MEALS_ORDERED +
Outlier_WEEKLY_PLAN +
Outlier_TOTAL_PHOTOS_VIEWED +
Outlier_UNIQUE_MEALS_PURCH +
Outlier_CONTACTS_W_CUSTOMER_SERVICE +
Outlier_CANCELLATIONS_BEFORE_NOON +
Outlier_EARLY_DELIVERIES +
Outlier_LATE_DELIVERIES +
Outlier_LARGEST_ORDER_SIZE +
Outlier_AVG_CLICKS_PER_VISIT +
Zero_CANCELLATIONS_BEFORE_NOON +
Zero_CANCELLATIONS_AFTER_NOON +
Zero_WEEKLY_PLAN +
Zero_TOTAL_PHOTOS_VIEWED +
Zero_EARLY_DELIVERIES +
Zero_LATE_DELIVERIES +
Zero_MASTER_CLASSES_ATTENDED +
log_Revenue +
log_CONTACTS_W_CUSTOMER_SERVICE +
log_Total_Meals_Ordered +
log_LARGEST_ORDER_SIZE +
log_AVG_CLICKS_PER_VISIT +
Single_Meals +
TOTAL_CLICKS_ratio +
TOTAL_LOGINS_all +
Total_Logins_per_Total_Meals_Ordered +
Product_Categories_Viewed_Per_TOTAL_LOGINS +
Unique_Meals_Purch_per_Total_Meals_Ordered +
Late_Delvery_per_Total_Meals +
Early_Delvery_per_Total_Meals +
All_Wrong_Deliveries +
Product_Categories_Viewed_per_Total_Logins +
Contacts_W_Customer_Service_per_Total_Orders +
personal +
professional
""",
                                     data = chef)


# telling Python to FIT the data to the blueprint
results = lm_practice.fit()


# printing a summary of the results
print(results.summary())

                            OLS Regression Results                            
Dep. Variable:     CROSS_SELL_SUCCESS   R-squared:                       0.167
Model:                            OLS   Adj. R-squared:                  0.142
Method:                 Least Squares   F-statistic:                     6.660
Date:                Wed, 27 Jan 2021   Prob (F-statistic):           6.89e-44
Time:                        02:15:09   Log-Likelihood:                -1100.9
No. Observations:                1946   AIC:                             2318.
Df Residuals:                    1888   BIC:                             2641.
Df Model:                          57                                         
Covariance Type:            nonrobust                                         
                                                   coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------------------------

In [63]:
#removing all variables with a p-value below 0.05 one by one.
#with this we can create a statistically significant OLS model 
#the result is shown below

#defining function to remove step by step variables wth a high p-value
def remove_insig(df, results):
    # use operator to find the key which belongs to the maximum value in the dictionary:
    p_max = max(results.pvalues.iteritems(), key=operator.itemgetter(1))[0]
    # this is the feature you want to drop:
    df.drop(columns = p_max, inplace = True)
    return chef #return the dataframe chef
#develop while loop
insig_feature = True
while insig_feature:
        model = sm.OLS(chef["CROSS_SELL_SUCCESS"], chef_explain) #set model
        results = model.fit() #apply fit function
        
        #create p-value boundaries (< 0.05) and with if statement remove high ones
        significant = [p_value < 0.05 for p_value in results.pvalues]
        if all(significant):
            insig_feature = False
        else:
            if chef_explain.shape[1] == 1:  # if there's only one insignificant variable left
                print('No significant features to be found')
                results = None
                insig_feature = False
            else:            
                X = remove_insig(chef_explain, results)

#print results of significant model
print(results.summary())

                                 OLS Regression Results                                
Dep. Variable:     CROSS_SELL_SUCCESS   R-squared (uncentered):                   0.728
Model:                            OLS   Adj. R-squared (uncentered):              0.726
Method:                 Least Squares   F-statistic:                              323.4
Date:                Wed, 27 Jan 2021   Prob (F-statistic):                        0.00
Time:                        02:15:10   Log-Likelihood:                         -1116.4
No. Observations:                1946   AIC:                                      2265.
Df Residuals:                    1930   BIC:                                      2354.
Df Model:                          16                                                  
Covariance Type:            nonrobust                                                  
                                           coef    std err          t      P>|t|      [0.025      0.975]
---------------

In [78]:
# preparing the explanatory variable data
chef_data = chef.drop(['CROSS_SELL_SUCCESS'], axis = 1)

# preparing the response variable
chef_response = chef.loc[ : , 'CROSS_SELL_SUCCESS']

In [79]:
#create dictionary after trying different approaches to see which one to select
data_columns_dict = {
    'significant' : 
         ['TASTES_AND_PREFERENCES', 'TOTAL_MEALS_ORDERED', 'CONTACTS_W_CUSTOMER_SERVICE', 
          'MOBILE_NUMBER','CANCELLATIONS_AFTER_NOON','CANCELLATIONS_BEFORE_NOON'
          ,'MOBILE_LOGINS','REFRIGERATED_LOCKER','log_Total_Meals_Ordered'
          ,'TOTAL_CLICKS_ratio','personal','professional','log_Revenue' ,'Total_Logins_per_Total_Meals_Ordered'
          ,'Zero_MASTER_CLASSES_ATTENDED','Outlier_CONTACTS_W_CUSTOMER_SERVICE'
          ,'Outlier_CANCELLATIONS_BEFORE_NOON',
         ]
}





In [80]:
# selecting the variables
chef_data   =  chef.loc[ :  , data_columns_dict['significant']]
chef_target =  chef.loc[ :  , 'CROSS_SELL_SUCCESS']


# non scaled train-test split
x_train, x_test, y_train, y_test = train_test_split(
            chef_data,
            chef_target,
            random_state = 219,
            test_size    = 0.25,
            stratify     = chef_target)


#SCALLED split
# INSTANTIATING StandardScaler()
scaler = StandardScaler()


# FITTING the data
scaler.fit(chef_data)


# TRANSFORMING the data
x_scaled     = scaler.transform(chef_data)


# converting to a DataFrame
x_scaled_df  = pd.DataFrame(x_scaled) 


# train-test split with the scaled data
x_train_scaled, x_test_scaled, y_train, y_test = train_test_split(
            x_scaled_df,
            chef_target,
            random_state = 219,
            test_size = 0.25,
            stratify = chef_target)

In [81]:
# INSTANTIATING a logistic regression model
logreg = LogisticRegression(solver = 'lbfgs',
                            C = 1,
                            random_state = 219)


# FITTING the training data
logreg_fit = logreg.fit(x_train_scaled, y_train)


# PREDICTING based on the testing set
logreg_pred = logreg_fit.predict(x_test_scaled)


# SCORING the results
print('LogReg Training ACCURACY:', logreg_fit.score(x_train_scaled, y_train).round(4))
print('LogReg Testing  ACCURACY:', logreg_fit.score(x_test_scaled, y_test).round(4))

# saving scoring data for future use
logreg_train_score = logreg_fit.score(x_train_scaled, y_train).round(4) # accuracy
logreg_test_score  = logreg_fit.score(x_test_scaled, y_test).round(4)   # accuracy


# displaying and saving the gap between training and testing
print('LogReg Train-Test Gap   :', abs(logreg_train_score - logreg_test_score).round(4))
logreg_test_gap = abs(logreg_train_score - logreg_test_score).round(4)

LogReg Training ACCURACY: 0.7265
LogReg Testing  ACCURACY: 0.7331
LogReg Train-Test Gap   : 0.0066


In [82]:
# unpacking the confusion matrix
logreg_tn, \
logreg_fp, \
logreg_fn, \
logreg_tp = confusion_matrix(y_true = y_test, y_pred = logreg_pred).ravel()


# printing each result one-by-one
print(f"""
True Negatives : {logreg_tn}
False Positives: {logreg_fp}
False Negatives: {logreg_fn}
True Positives : {logreg_tp}
""")


True Negatives : 59
False Positives: 97
False Negatives: 33
True Positives : 298



In [83]:
# area under the roc curve (auc)
print(roc_auc_score(y_true  = y_test,
                    y_score = logreg_pred).round(decimals = 4))


# saving AUC score for future use
logreg_auc_score = roc_auc_score(y_true  = y_test,
                                 y_score = logreg_pred).round(decimals = 4)

0.6393


In [84]:
# INSTANTIATING a classification tree object
full_tree = DecisionTreeClassifier()


# FITTING the training data
full_tree_fit = full_tree.fit(chef_data,chef_target)


# PREDICTING on new data
full_tree_pred = full_tree_fit.predict(x_test)


# SCORING the model
print('Full Tree Training ACCURACY:', full_tree_fit.score(x_train,
                                                    y_train).round(4))

print('Full Tree Testing ACCURACY :', full_tree_fit.score(x_test,
                                                    y_test).round(4))

print('Full Tree AUC Score:', roc_auc_score(y_true  = y_test,
                                            y_score = full_tree_pred).round(4))


# saving scoring data for future use
full_tree_train_score = full_tree_fit.score(x_train, y_train).round(4) # accuracy
full_tree_test_score  = full_tree_fit.score(x_test, y_test).round(4)   # accuracy


# saving AUC
full_tree_auc_score   = roc_auc_score(y_true  = y_test,
                                      y_score = full_tree_pred).round(4) # auc

Full Tree Training ACCURACY: 1.0
Full Tree Testing ACCURACY : 1.0
Full Tree AUC Score: 1.0


In [85]:
# unpacking the confusion matrix
full_tree_tn, \
full_tree_fp, \
full_tree_fn, \
full_tree_tp = confusion_matrix(y_true = y_test, y_pred = full_tree_pred).ravel()


# printing each result one-by-one
print(f"""
True Negatives : {full_tree_tn}
False Positives: {full_tree_fp}
False Negatives: {full_tree_fn}
True Positives : {full_tree_tp}
""")


True Negatives : 156
False Positives: 0
False Negatives: 0
True Positives : 331



In [86]:
# INSTANTIATING a classification tree object
tree_pruned = DecisionTreeClassifier(max_depth        = 8,
                                     min_samples_leaf = 25,
                                     random_state     = 219)


# FITTING the training data
pruned_tree_fit = tree_pruned.fit(chef_data, chef_target)


# PREDICTING on new data
pruned_tree_pred = pruned_tree_fit.predict(x_test)


# SCORING the model
print('Training ACCURACY:', pruned_tree_fit.score(x_train, y_train).round(4))
print('Testing  ACCURACY:', pruned_tree_fit.score(x_test, y_test).round(4))
print('AUC Score        :', roc_auc_score(y_true  = y_test,
                                          y_score = pruned_tree_pred).round(4))


# saving scoring data for future use
pruned_tree_train_score = pruned_tree_fit.score(x_train, y_train).round(4) # accuracy
pruned_tree_test_score  = pruned_tree_fit.score(x_test, y_test).round(4) # accuracy


# saving auc score
pruned_tree_auc_score   = roc_auc_score(y_true  = y_test,
                                        y_score = pruned_tree_pred).round(4) # auc

Training ACCURACY: 0.7464
Testing  ACCURACY: 0.7577
AUC Score        : 0.6726


In [87]:
# unpacking the confusion matrix
pruned_tree_tn, \
pruned_tree_fp, \
pruned_tree_fn, \
pruned_tree_tp = confusion_matrix(y_true = y_test, y_pred = pruned_tree_pred).ravel()


# printing each result one-by-one
print(f"""
True Negatives : {pruned_tree_tn}
False Positives: {pruned_tree_fp}
False Negatives: {pruned_tree_fn}
True Positives : {pruned_tree_tp}
""")


True Negatives : 68
False Positives: 88
False Negatives: 30
True Positives : 301



In [88]:
# declaring a hyperparameter space
criterion_space = ['gini', 'entropy']
splitter_space  = ['best', 'random']
depth_space     = pd.np.arange(1, 25, 1)
leaf_space      = pd.np.arange(1, 100, 1)


# creating a hyperparameter grid
param_grid = {'criterion'        : criterion_space,
              'splitter'         : splitter_space,
              'max_depth'        : depth_space,
              'min_samples_leaf' : leaf_space}


# INSTANTIATING the model object without hyperparameters
tuned_tree = DecisionTreeClassifier(random_state = 219)


# RandomizedSearchCV object
tuned_tree_cv = RandomizedSearchCV(estimator             = tuned_tree,
                                   param_distributions   = param_grid,
                                   cv                    = 3,
                                   n_iter                = 1000,
                                   random_state          = 219,
                                   scoring = make_scorer(roc_auc_score,
                                             needs_threshold = False))






# FITTING to the FULL chef (due to cross-validation)
model_fit=tuned_tree_cv.fit(chef_data, chef_target)




model_pred = tuned_tree_cv.predict(x_test)


# printing the optimal parameters and best score
print("Tuned Parameters  :", tuned_tree_cv.best_params_)
print("Tuned Training AUC:", tuned_tree_cv.best_score_.round(4))

<ipython-input-88-c70ad94db598>:4: FutureWarning: The pandas.np module is deprecated and will be removed from pandas in a future version. Import numpy directly instead
  depth_space     = pd.np.arange(1, 25, 1)
<ipython-input-88-c70ad94db598>:5: FutureWarning: The pandas.np module is deprecated and will be removed from pandas in a future version. Import numpy directly instead
  leaf_space      = pd.np.arange(1, 100, 1)


Tuned Parameters  : {'splitter': 'random', 'min_samples_leaf': 16, 'max_depth': 9, 'criterion': 'entropy'}
Tuned Training AUC: 0.635


In [93]:
# building a model based on hyperparameter tuning results

# INSTANTIATING a logistic regression model with tuned values
tree_tuned = {'splitter': 'random', 'min_samples_leaf': 16, 'max_depth': 9, 'criterion': 'entropy'}


# FIT step is not needed
model_fit=tree_tuned.fit(chef_data, chef_target)


# PREDICTING based on the testing set
tree_tuned_pred = tree_tuned.predict(x_test)


# SCORING the results
print('Training ACCURACY:', tree_tuned.score(x_train, y_train).round(4))
print('Testing  ACCURACY:', tree_tuned.score(x_test, y_test).round(4))
print('AUC Score        :', roc_auc_score(y_true  = y_test,
                                          y_score = tree_tuned_pred).round(4))


# saving scoring data for future use
tree_tuned_train_score = tree_tuned.score(x_train, y_train).round(4) # accuracy
tree_tuned_test_score  = tree_tuned.score(x_test, y_test).round(4)   # accuracy


# saving the AUC score
tree_tuned_auc         = roc_auc_score(y_true  = y_test,
                                     y_score = tree_tuned_pred).round(4) # auc

AttributeError: 'dict' object has no attribute 'fit'

In [92]:
# display_tree

def display_tree(tree, feature_df, height = 500, width = 800):
    """
    PARAMETERS
    ----------
    tree       : fitted tree model object
        fitted CART model to visualized
    feature_df : DataFrame
        DataFrame of explanatory features (used to generate labels)
    height     : int, default 500
        height in pixels to which to constrain image in html
    width      : int, default 800
        width in pixels to which to constrain image in html
    """

    # visualizing the tree
    dot_data = StringIO()

    
    # exporting tree to graphviz
    export_graphviz(decision_tree      = tree,
                    out_file           = dot_data,
                    filled             = True,
                    rounded            = True,
                    special_characters = True,
                    feature_names      = feature_df.columns)


    # declaring a graph object
    graph = pydotplus.graph_from_dot_data(dot_data.getvalue())


    # creating image
    img = Image(graph.create_png(),
                height = height,
                width  = width)
    
    return img

# plot_feature_importances

def plot_feature_importances(model, train, export = False):
    """
    Plots the importance of features from a CART model.
    
    PARAMETERS
    ----------
    model  : CART model
    train  : explanatory variable training data
    export : whether or not to export as a .png image, default False
    """
    
    # declaring the number
    n_features = x_train.shape[1]
    
    # setting plot window
    fig, ax = plt.subplots(figsize=(12,9))
    
    plt.barh(range(n_features), model.feature_importances_, align='center')
    plt.yticks(pd.np.arange(n_features), train.columns)
    plt.xlabel("Feature importance")
    plt.ylabel("Feature")
    
    if export == True:
        plt.savefig('Tree_Leaf_50_Feature_Importance.png')
        plt.show()

In [77]:
# comparing results
print(f"""
Model         AUC Score      TN, FP, FN, TP
-----         ---------      --------------
Logistic      {logreg_auc_score}         {logreg_tn, logreg_fp, logreg_fn, logreg_tp}
Full Tree     {full_tree_auc_score}         {full_tree_tn, full_tree_fp, full_tree_fn, full_tree_tp}
Pruned Tree   {pruned_tree_auc_score}         {pruned_tree_tn, pruned_tree_fp, pruned_tree_fn, pruned_tree_tp}
""")


# creating a dictionary for model results
model_performance = {
    
    'Model Name'    : ['Logistic', 'Full Tree', 'Pruned Tree'],
           
    'AUC Score' : [logreg_auc_score, full_tree_auc_score, pruned_tree_auc_score],
    
    'Training Accuracy' : [logreg_train_score, full_tree_train_score,
                           pruned_tree_train_score],
           
    'Testing Accuracy'  : [logreg_test_score, full_tree_test_score,
                           pruned_tree_test_score],

    'Confusion Matrix'  : [(logreg_tn, logreg_fp, logreg_fn, logreg_tp),
                           (full_tree_tn, full_tree_fp, full_tree_fn, full_tree_tp),
                           (pruned_tree_tn, pruned_tree_fp, pruned_tree_fn, pruned_tree_tp)]}


# converting model_performance into a DataFrame
model_performance = pd.DataFrame(model_performance)


# sending model results to Excel
model_performance.to_excel('./model_results/classification_model_performance.xlsx',
                           index = False)


Model         AUC Score      TN, FP, FN, TP
-----         ---------      --------------
Logistic      0.6393         (59, 97, 33, 298)
Full Tree     1.0         (156, 0, 0, 331)
Pruned Tree   0.6726         (68, 88, 30, 301)



In [ ]:
data   =  x_variables

target =  y_variable

 

x_train, x_test, y_train, y_test = train_test_split(

            data,

            target_target,

            random_state = 219,

            test_size    = 0.25,

            stratify     =  target)

 

# copy/pasting in the best_estimator_ results

model = RandomForestClassifier( hyperparameter tuning results )

 

# FITTING the model object

model_fit = forest_tuned.fit(x_train, y_train) # the mistake in the code

 

# PREDICTING based on the testing set

model_pred = forest_tuned_fit.predict(x_test)

 

######################################################################

 

 

######################################################################

## Correction to get better AUC Results

######################################################################

 

# copy/pasting in the best_estimator_ results

model = RandomForestClassifier( hyperparameter tuning results )

 

# FITTING the model object

model_fit = forest_tuned.fit(data, target)  # full set created right before train_test_split

 

# PREDICTING based on the testing set

model_pred = forest_tuned_fit.predict(x_test)

 